# Operating Systems: processes
_COSC 208, Introduction to Computer Systems, Spring 2025_

## Announcements
* Project 3 intial submission due Tuesday, April 15 @ 11pm

## Process abstraction

* Process — running program and its corresponding machine state (code, memory, and register values)
    * Program is static code and static data; process is dynamic instance of code and data
    * Cooking analogy
        * Recipe == program
        * Following a recipe == process
        * Ingredients == program inputs
        * Prepared food == program outputs
        * Cabinets, fridge, etc. == persistent storage
        * Prep area (e.g., counter) == registers & main memory
        * Contents and status of the prep area; current step of recipe == machine state
        * Chef == CPU
    * Can have multiple processes all running different instances of the same program
        * Cooking analogy — chef may be making multiple batches of the same recipe
* OS is responsible for...
    * Creating processes — when a user or another application requests it
    * Scheduling processes — i.e., deciding when/which process should be allowed to use the CPU
    * Switching processes — i.e., saving the machine state of one process and restoring the machine state of another process; called context switching
    * Cleaning-up processes — i.e., releasing any resources they are using when the process is done
    * Interacting with I/O devices (e.g., disks, NICs, graphics card) on behalf of processes

<div style="height:11em;"></div>

Q1: _Consider building a Lego kit as an analogy for operating systems' process abstraction. Match each component of the analogy with the corresponding component of a real computer system._

* Analogy
    1. Cabinet/drawers for storing Legos
    2. Lego bricks
    3. Building area (e.g., tabletop)
    4. Instruction booklet
    5. Following the assembly instructions
    6. Current step for the instruction booklet
    7. Completed kit
    8. You
* Real system
    * CPU
    * persistent storage
    * process
    * program
    * program counter
    * program inputs
    * program outputs
    * registers and main memory

* Instruction booklet == program
* Following the assembly instructions == process
* Lego bricks == program inputs
* Completed kit == program outputs
* Cabinet/drawers for storing Legos == persistent storage
* Building area (e.g., tabletop) == registers and main memory
* Current step for the instruction booklet == program counter
* You == CPU

## Creating processes

* `int fork()`
    * Creates an exact copy of the running process, except for the return value from `fork` — return `0` to child (i.e., new) process; return child’s process ID to parent process (i.e., process that called fork)
    * Both child and parent resume execution from place where `fork` was called

<div style="height:5em;"></div>

Q2: _What does the following code output?_

In [1]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main(int argc, char **argv) {
/* 4*/      int x = 13;
/* 5*/      printf("Before fork %d\n", x);
/* 6*/      fork();
/* 7*/      printf("After fork %d\n", x);
/* 8*/      return 0;
/* 9*/  }

Before fork 13
After fork 13
After fork 13


<div style="height:10em;"></div>

Q3: _What does the following code output (assuming the new process has PID 1819)?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>    
/* 3*/  int main(int argc, char **argv) {
/* 4*/      printf("Before fork\n");
/* 5*/      int pid = fork();
/* 6*/      if (pid < 0) {
/* 7*/          printf("Fork failed\n");
/* 8*/          return 1;
/* 9*/      } else if (pid == 0) {
/*10*/          printf("Child gets %d\n", pid);
/*11*/      } else {
/*12*/          printf("Parent gets %d\n", pid);
/*13*/      }
/*14*/      return 0;
/*15*/  }

```
Before fork
Child gets 0
Parent gets 1819
```
OR
```
Before fork
Parent gets 1819
Child gets 0
``` 

<div style="height:10em;"></div>

Q4: _What are all possible outputs of this program, assuming the process identifier (PID) of the parent process is 50 and the PID of the child process is 60?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>    
/* 3*/  int main() {
/* 4*/      int n = 20;
/* 5*/      int x = fork();
/* 6*/      if (x < 0) {
/* 7*/          printf("Fork failed\n");
/* 8*/          return 1;
/* 9*/      } else if (x == 0) {
/*10*/          n += 5;
/*11*/      } else {
/*12*/          n += 10;
/*13*/      }
/*14*/      printf("x=%d n=%d\n", x, n);
/*15*/  }

```
x=0 n=25
x=60 n=30
```
OR
```
x=60 n=30
x=0 n=25
```

<div style="height:9em;"></div>

Q5: _What does the following code output?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main() {
/* 4*/      int x = 10;
/* 5*/      int y = 20;
/* 6*/      int retval = fork();
/* 7*/      if (retval < 0) {
/* 8*/          printf("Fork failed\n");
/* 9*/          return 1;
/*10*/      } else if (retval > 0) {
/*11*/          y+= 5;
/*12*/      } else {
/*13*/          y -= 5;
/*14*/      }
/*15*/      printf("x=%d y=%d\n", x, y);
/*16*/      return 0;
/*17*/  }

```
x=10 y=25
x=10 y=15
```
OR
```
x=10 y=15
x=10 y=25
```

<div style="height:9em;"></div>

Q6: _What does the following code output?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main(int argc, char **argv) {
/* 4*/      int value = 100;
/* 5*/      int pid = fork();
/* 6*/      if (pid < 0) {
/* 7*/          printf("Fork failed\n");
/* 8*/          return 1;
/* 9*/      } else if (pid > 0) {
/*10*/          value -= 50;
/*11*/      } else {
/*12*/          value += 50;
/*13*/      }
/*14*/      printf("My value is %d\n", value);
/*15*/      return 0;
/*16*/  }

```
My value is 50
My value is 150
```
OR
```
My value is 150
My value is 50
```

<p style="height:10em;"></p>

## Waiting for processes

* Wait for any child to finish — `int wait(int *status)`
    * Returns PID of the child process that finished
    * `status` parameter is optional
        * if passed a valid integer pointer, `wait` will store the return value of the child process's `main` function at the referenced memory location
        * if return value of child process's main function is not needed, then pass `NULL`
* Wait for a specific process to finish — `int waitpid(pid_t pid, int *status, int options)`
    * Returns PID of the process that finished
    * `pid` is PID if process to wait for — need not be a child process
    * `status` is the same as `wait`
    * `options` is typically `0`, except in special circumstances
* Wait functions do not return until child or specific process, respectively, finishes

Q7: _What are all possible outputs of this program?_

In [7]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  #include <sys/wait.h>
/* 4*/  int main() {
/* 5*/      int pid = fork();
/* 6*/      if (pid < 0) {
/* 7*/          printf("Fork failed\n");
/* 8*/          return 1;
/* 9*/      } else if (pid > 0) {
/*10*/          wait(NULL);
/*11*/          printf("Parent\n");
/*12*/      } else {
/*13*/          printf("Child\n");
/*14*/      }
/*15*/      return 0;
/*16*/  }

Child
Parent


<div style="height:6em;"></div>

Q8: _How would you modify the program below such that `Chicken` always prints before `Egg`?_

In [ ]:
#include <stdio.h>
#include <unistd.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid > 0) {
        printf("Egg\n");
        return 0;
    } else {
        printf("Chicken\n");
        return 0;
    }
}

In [ ]:
#include <stdio.h>
#include <unistd.h>
#include <sys/wait.h>
int main(int argc, char **argv) {
    printf("Begin\n");
    int pid = fork();
    if (pid < 0) {
        printf("Fork failed\n");
        return 1;
    } else if (pid > 0) {
        wait(NULL);
        printf("Egg\n");
        return 0;
    } else {
        printf("Chicken\n");
        return 0;
    }
}

<div style="height:6em;"></div>

Q9: _What are all possible outputs of this program (assuming the new process has PID 13346)?_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  #include <sys/wait.h>
/* 4*/  int main() {
/* 5*/      int pid = fork();
/* 6*/      printf("A %d\n", pid);
/* 7*/      if (pid == 0) {
/* 8*/          printf("B\n");
/* 9*/      } else if (pid > 0) {
/*10*/          wait(NULL);
/*11*/          printf("C\n");
/*12*/      }
/*13*/  }

```
A 0
A 13346
B
C
```
OR
```
A 13346
A 0
B
C
```
OR
```
A 0
B
A 13346
C
```

<div style="height:12em;"></div>

Q10: _What are all possible outputs of this program_?

In [19]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <unistd.h>
/* 4*/  void *proc1_main(void *arg) {
/* 5*/      int *x = (int *)arg;
/* 6*/      *x += 1;
/* 7*/      return NULL;
/* 8*/  }
/* 9*/  void *proc2_main(void *arg) {
/*10*/      int *y = (int *)arg;
/*11*/      *y += 2;
/*12*/      return NULL;
/*13*/  }
/*14*/  int main() {
/*15*/      int *z = malloc(sizeof(int));
/*16*/      *z = 0;
/*17*/      int pid = fork();
/*18*/      if (pid == 0) {
/*19*/          proc1_main(z);
/*20*/      } else {
/*21*/          wait(NULL);
/*22*/          proc2_main(z);
/*23*/      }
/*24*/      printf("z is %d\n", *z);
/*25*/  }

z is 1
z is 2


<div style="height:1em;"></div>

Q11: _What output is produced by the following program? (Note: there is only one possible ordering.)_

In [ ]:
/*1*/   #include <stdio.h>
/*2*/   #include <stdlib.h>
/*3*/   #include <sys/wait.h>
/*4*/   #include <unistd.h>
/*5*/   int main() {
/*6*/       printf("A\n");
/*7*/       int x = fork();
/*8*/       if (x == 0) {
/*9*/           int y = fork();
/*10*/          if (y == 0) {
/*11*/              printf("B\n");
/*12*/          }
/*13*/          else {
/*14*/              wait(NULL);
/*15*/              printf("C\n");
/*16*/          }
/*17*/      }
/*18*/      else {
/*19*/          wait(NULL);
/*20*/          printf("D\n");
/*21*/      }
/*22*/      printf("E\n");
/*23*/  }

A
B
E
C
E
D
E


<p style="height:1em;"></p>

Q12: _What output is produced by this program, assuming the starting process's PID is 7034 and PIDs are sequentially assigned to new processes. (Note: there is only one possible ordering.)_

In [ ]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  int main(int argc, char **argv) {
/* 4*/      printf("Begin\n");
/* 5*/      int result = fork();
/* 6*/      if (result == 0) {
/* 7*/          printf("A %d %d\n", result, getpid());
/* 8*/      } else if (result > 0) {
/* 9*/          wait(NULL);
/*10*/          printf("B %d %d\n", result, getpid());
/*11*/      }
/*12*/      int result2 = fork();
/*13*/      if (result2 == 0) {
/*14*/          printf("C %d %d\n", result2, getpid());
/*15*/      } else if (result2 > 0) {
/*16*/          wait(NULL);
/*17*/          printf("D %d %d\n", result2, getpid());
/*18*/      }
/*19*/  }

```
Begin
A 0 7035
C 0 7036
D 7036 7035
B 7035 7034
C 0 7037
D 7037 7034
```

<p style="height:7em;"></p>

## Running a different program

_Example program_

In [11]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  #include <sys/wait.h>
/* 4*/  int main(int argc, char **argv) {
/* 5*/      printf("Begin\n");
/* 6*/      int pid = fork();
/* 7*/      if (pid < 0) {
/* 8*/          printf("Fork failed\n");
/* 9*/          return 1;
/*10*/      } else if (pid == 0) {
/*11*/          printf("Child\n");
/*12*/          char *cmd[] = { "/bin/date", NULL };
/*13*/          execv(cmd[0], cmd);
/*14*/      } else {
/*15*/          wait(NULL);
/*16*/          printf("Parent\n");
/*17*/      }
/*18*/      printf("End\n");
/*19*/      return 0;
/*20*/  }

Begin
Child
Mon Nov 11 14:54:21 EST 2024
Parent
End


<p style="height:7em;"></p>

Q13: _What is the output produced by running `./progA`, assuming no errors occur?_

In [ ]:
/** progA **/
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  #include <sys/wait.h>
/* 4*/  int main() {
/* 5*/      int a = fork();
/* 6*/      if (a == 0) {
/* 7*/          char *cmd[] = { "./progB", NULL };
/* 8*/          execv(cmd[0], cmd);
/* 9*/          printf("A 2nd gen\n");
/*10*/          return 0;
/*11*/      } else {
/*12*/          wait(NULL);
/*13*/          printf("A 1st gen\n");
/*14*/          return 0;
/*15*/      }
/*16*/  }

In [ ]:
/** progB **/
/* 1*/  #include <stdio.h>
/* 2*/  #include <unistd.h>
/* 3*/  #include <sys/wait.h>
/* 4*/  int main() {
/* 5*/      int b = fork();
/* 6*/      if (b == 0) {
/* 7*/          printf("B 2nd gen\n");
/* 8*/          return 0;
/* 9*/      } else {
/*10*/          wait(NULL);
/*11*/          printf("B 1st gen\n");
/*12*/          return 0;
/*13*/      }
/*14*/  }

    B 2nd gen
    B 1st gen
    A 1st gen

<p style="height:5em;"></p>